In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute

In [2]:
DATA_PATH = '../data/'
FEATURE_PATH = '../'
train_log_df = pd.read_csv(os.path.join(DATA_PATH, 'train_log.csv'))
metadata = train_log_df[['object_id', 'Z', 'EBV', 'target']]
print("Metadata loaded successfully")

Metadata loaded successfully


In [3]:
# Load ALL Lightcurve Data for tsFresh
all_lc_df_list = []
print("Loading all lightcurve data...")
for split_folder in tqdm(train_log_df['split'].unique()):
    path = os.path.join(DATA_PATH, split_folder, 'train_full_lightcurves.csv')
    df = pd.read_csv(path)
    all_lc_df_list.append(df)

full_lc_df = pd.concat(all_lc_df_list)
print(f"Full lightcurve data loaded. Shape: {full_lc_df.shape}")

Loading all lightcurve data...


100%|██████████| 20/20 [00:00<00:00, 96.65it/s]

Full lightcurve data loaded. Shape: (479384, 5)


In [ ]:
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import EfficientFCParameters


# Feature Extraction with tsFresh
print("Starting tsFresh feature extraction per filter...")

band_features = []
for f in full_lc_df['Filter'].unique():
    subset = full_lc_df[full_lc_df['Filter'] == f][['object_id', 'Time (MJD)', 'Flux']].copy()
    subset.columns = ['id', 'time', 'value']
    
    # Drop NaNs
    subset = subset.dropna(subset=['value'])
    
    feats = extract_features(
        subset,
        column_id='id',
        column_sort='time',
        default_fc_parameters=EfficientFCParameters(),
        disable_progressbar=False,
        n_jobs=4
    )
    # tag features with filter name
    feats.columns = [f"{col}_{f}" for col in feats.columns]
    band_features.append(feats)

# Merge all filter-specific features
extracted_features = pd.concat(band_features, axis=1)

# Impute NaN/Infs
impute(extracted_features)

# Feature selection
y_for_selection = metadata.set_index('object_id').loc[extracted_features.index]['target']
relevant_features = select_features(
    extracted_features,
    y_for_selection,
    fdr_level=0.005
)

print(f"tsFresh finished successfully. Found {relevant_features.shape[1]} relevant features.")


Starting tsFresh feature extraction per filter...


Feature Extraction: 100%|██████████| 20/20 [00:30<00:00,  1.51s/it]
c:\Project\MALLORN Astronomical Classification Challenge\.venv\Lib\site-packages\tsfresh\utilities\dataframe_functions.py:198: RuntimeWarning: The columns ['value__query_similarity_count__query_None__threshold_0.0_z'
 'value__query_similarity_count__query_None__threshold_0.0_r'
 'value__fft_coefficient__attr_"real"__coeff_93_y'
 'value__fft_coefficient__attr_"real"__coeff_94_y'
 'value__fft_coefficient__attr_"real"__coeff_95_y'
 'value__fft_coefficient__attr_"real"__coeff_96_y'
 'value__fft_coefficient__attr_"real"__coeff_97_y'
 'value__fft_coefficient__attr_"real"__coeff_98_y'
 'value__fft_coefficient__attr_"real"__coeff_99_y'
 'value__fft_coefficient__attr_"imag"__coeff_93_y'
 'value__fft_coefficient__attr_"imag"__coeff_94_y'
 'value__fft_coefficient__attr_"imag"__coeff_95_y'
 'value__fft_coefficient__attr_"imag"__coeff_96_y'
 'value__fft_coefficient__attr_"imag"__coeff_97_y'
 'value__fft_coefficient__attr_"imag"__co

tsFresh finished successfully. Found 198 relevant features.


In [5]:
# Combine Features and Prepare for Modeling
# Marge the new tsFresh with best old features
final_features = metadata.merge( 
    relevant_features,
    left_on='object_id',
    right_index=True,
    how='inner'
)

# Clean column names 
final_features.columns = (
    final_features.columns
    .str.replace('[^A-Za-z0-9_]+', '_', regex=True)  
    .str.strip('_')                                  
)

# Prepare X and y
y = final_features['target']
X = final_features.drop(columns=['object_id', 'target'])
features = X.columns.tolist()

print(f"Data prepared for final modeling. New features shape: {X.shape}")

Data prepared for final modeling. New features shape: (3043, 200)


In [ ]:
import optuna
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd

# Standard scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X = pd.DataFrame(X_scaled, columns=X.columns)

# Cross-validation
N_SPLITS = 5
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

# Objective function for Optuna
def objective(trial):
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'n_estimators': 2000,
        'device': 'gpu',
        'gpu_platform_id': 0,
        'gpu_device_id': 0,
        'verbose': -1
    }

    f1_scores = []

    for train_idx, val_idx in skf.split(X, y):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

        # Compute scale_pos_weight
        neg_count = y_train.value_counts()[0]
        pos_count = y_train.value_counts()[1]
        params['scale_pos_weight'] = neg_count / pos_count

        model = lgb.LGBMClassifier(**params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            eval_metric='f1',
            callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)]
        )

        val_preds_proba = model.predict_proba(X_val)[:, 1]
        thresholds = np.linspace(0.01, 0.99, 100)
        f1_values = [f1_score(y_val, (val_preds_proba > t).astype(int)) for t in thresholds]
        f1_scores.append(np.max(f1_values))

    return np.mean(f1_scores)

# Run Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("\nBest F1 Score:", study.best_value)
print("Best Hyperparameters:", study.best_params)


c:\Project\MALLORN Astronomical Classification Challenge\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-11-05 22:39:33,342] A new study created in memory with name: no-name-95b383fc-2644-4630-893a-2d6845f7e048
Best trial: 0. Best value: 0.432581:   2%|▏         | 1/50 [00:04<03:49,  4.68s/it]

[I 2025-11-05 22:39:38,017] Trial 0 finished with value: 0.43258134763253386 and parameters: {'learning_rate': 0.018982483517193772, 'num_leaves': 30, 'max_depth': 15, 'min_child_samples': 98, 'subsample': 0.5305209484325696, 'colsample_bytree': 0.8796427422742497, 'reg_alpha': 0.4465506234330048, 'reg_lambda': 0.36768705889218567}. Best is trial 0 with value: 0.43258134763253386.


Best trial: 1. Best value: 0.50221:   4%|▍         | 2/50 [00:11<04:52,  6.09s/it] 

[I 2025-11-05 22:39:45,093] Trial 1 finished with value: 0.5022101283234087 and parameters: {'learning_rate': 0.06668892133207022, 'num_leaves': 87, 'max_depth': 9, 'min_child_samples': 20, 'subsample': 0.6909592547843314, 'colsample_bytree': 0.6047502160310755, 'reg_alpha': 0.5785348467411694, 'reg_lambda': 0.0972102414877698}. Best is trial 1 with value: 0.5022101283234087.


Best trial: 1. Best value: 0.50221:   6%|▌         | 3/50 [00:13<03:15,  4.17s/it]

...

[I 2025-11-05 22:44:56,793] Trial 47 finished with value: 0.5142820745497696 and parameters: {'learning_rate': 0.08679873011594914, 'num_leaves': 50, 'max_depth': 8, 'min_child_samples': 93, 'subsample': 0.5713005589659338, 'colsample_bytree': 0.6130844156339919, 'reg_alpha': 0.1537483138746195, 'reg_lambda': 0.053041808829233816}. Best is trial 22 with value: 0.5224743283501228.


Best trial: 22. Best value: 0.522474:  98%|█████████▊| 49/50 [05:28<00:05,  5.63s/it]

[I 2025-11-05 22:45:01,691] Trial 48 finished with value: 0.4893768444155082 and parameters: {'learning_rate': 0.09198124726597347, 'num_leaves': 33, 'max_depth': 8, 'min_child_samples': 96, 'subsample': 0.6734114660446525, 'colsample_bytree': 0.6911679265124437, 'reg_alpha': 0.14686076970180853, 'reg_lambda': 0.0590142215710925}. Best is trial 22 with value: 0.5224743283501228.


Best trial: 22. Best value: 0.522474: 100%|██████████| 50/50 [05:33<00:00,  6.67s/it]

[I 2025-11-05 22:45:06,949] Trial 49 finished with value: 0.48022113023558866 and parameters: {'learning_rate': 0.08671533232626244, 'num_leaves': 51, 'max_depth': 9, 'min_child_samples': 91, 'subsample': 0.5654115192623522, 'colsample_bytree': 0.7702552879089403, 'reg_alpha': 0.07223439763660038, 'reg_lambda': 0.007440693144257993}. Best is trial 22 with value: 0.5224743283501228.

Best F1 Score: 0.5224743283501228
Best Hyperparameters: {'learning_rate': 0.03610120340507472, 'num_leaves': 120, 'max_depth': 11, 'min_child_samples': 80, 'subsample': 0.5577339749020074, 'colsample_bytree': 0.5736019643417907, 'reg_alpha': 0.31579743846984376, 'reg_lambda': 0.3170517208465995}
